In [134]:
# user inputs
userInput = "Ik ben Sam, een tweedejaars hbo-student met een sterke interesse in data, technologie en creatief ontwerp. Ik vind het leuk om bezig te zijn met projecten waarin ik kan experimenteren, zoals prototyping, data-analyse of het bouwen van digitale oplossingen. Daarnaast wil ik graag meer leren over duurzaamheid en hoe technologie kan bijdragen aan maatschappelijke vraagstukken. Ik studeer in Breda, maar modules in Den Bosch zijn voor mij ook prima bereikbaar. Voor de komende periode wil ik ongeveer 30 studiepunten volgen. Ik zoek vooral modules met een gemiddelde moeilijkheidsgraad — ik wil iets leren dat me uitdaagt, maar het moet wel haalbaar blijven. Populariteit vind ik minder belangrijk; voor mij telt vooral of een module past bij mijn interesses en persoonlijke ontwikkeling."

In [135]:
import re
import nltk
from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer, PorterStemmer
# from gensim.models import Word2Vec
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Download required NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

df_import = pd.read_csv("Uitgebreide_VKM_dataset_cleaned2.csv")

df = df_import.copy()
df_test = df_import.copy()
#to lower case
lowerd_input = userInput.lower()
print("Cleaned Text:\n", lowerd_input)

# Remove numbers
# text_no_numbers = re.sub(r'\d+', '',text_clean )
# print("Cleaned Text:\n", text_clean)

Cleaned Text:
 ik ben sam, een tweedejaars hbo-student met een sterke interesse in data, technologie en creatief ontwerp. ik vind het leuk om bezig te zijn met projecten waarin ik kan experimenteren, zoals prototyping, data-analyse of het bouwen van digitale oplossingen. daarnaast wil ik graag meer leren over duurzaamheid en hoe technologie kan bijdragen aan maatschappelijke vraagstukken. ik studeer in breda, maar modules in den bosch zijn voor mij ook prima bereikbaar. voor de komende periode wil ik ongeveer 30 studiepunten volgen. ik zoek vooral modules met een gemiddelde moeilijkheidsgraad — ik wil iets leren dat me uitdaagt, maar het moet wel haalbaar blijven. populariteit vind ik minder belangrijk; voor mij telt vooral of een module past bij mijn interesses en persoonlijke ontwikkeling.


[nltk_data] Downloading package stopwords to C:\Users\Dhr.
[nltk_data]     Kootwijk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Dhr.
[nltk_data]     Kootwijk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Dhr.
[nltk_data]     Kootwijk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Dhr.
[nltk_data]     Kootwijk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [136]:
sample_size = 200
df = df.sample(n=sample_size, replace=False, random_state=490)

df = df.reset_index()
df = df.drop('index',axis=1)

In [137]:
def clean_text(author):
    result = str(author).lower()
    return(result.replace(' ',''))

df['name'] = df['name'].apply(clean_text)
df['name'] = df['name'].str.lower()
# df['Publisher'] = df['Publisher'].str.lower()

In [138]:
# combine all strings:
df2 = df.drop(['shortdescription', 'content', 'description', 'learningoutcomes'],axis=1)

df2['data'] = df2[df2.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
print(df2['data'].head())

0    factoryofthefuture 30 Den Bosch 101 NLQF6 ['sm...
1    didactiekvoorn&genn&t 15 Breda 101 NLQF6 ['did...
2    seriousgaming-optie2* 15 Den Bosch 101 NLQF5 [...
3    advancedcomputervision 15 Breda 101 NLQF6 ['sm...
4    kind,kraam,jeugd 30 Breda 76 NLQF6 ['jeugd', '...
Name: data, dtype: object


In [139]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorized = vectorizer.fit_transform(df2['data'])

from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vectorized)
print(similarities)


[[1.         0.18190172 0.22941573 ... 0.1767767  0.19364917 0.1767767 ]
 [0.18190172 1.         0.16692447 ... 0.17149859 0.31311215 0.28583098]
 [0.22941573 0.16692447 1.         ... 0.10814761 0.23693955 0.27036904]
 ...
 [0.1767767  0.17149859 0.10814761 ... 1.         0.12171612 0.22222222]
 [0.19364917 0.31311215 0.23693955 ... 0.12171612 1.         0.24343225]
 [0.1767767  0.28583098 0.27036904 ... 0.22222222 0.24343225 1.        ]]


In [140]:
df = pd.DataFrame(similarities, columns=df['name'], index=df['name']).reset_index()

df.head()

name,name,factoryofthefuture,didactiekvoorn&genn&t,seriousgaming-optie2*,advancedcomputervision,"kind,kraam,jeugd",toekomstgerichteproductielijn,createandexhibit,molecularmodeling&data-drivenanalysis,developmentforstorytelling,...,chemicalmodelling,innovationforsustainability\r\n(sdginmotion),"digitalnomadsnlqf6,30+15ec",datadrivenbusiness,module4.1/4.2:infrastructuur(bestaandeminor:doorontwikkelen),arbeidsrecht,minoraquacycle:circularwatertechnology,innovationmanagement&creativeleadership,industriëlerobotcel,makerspace
0,factoryofthefuture,1.000000,0.181902,0.229416,0.416025,0.181902,0.363803,0.045644,0.196116,0.303170,...,0.353553,0.224507,0.279508,0.303170,0.266501,0.322749,0.381881,0.176777,0.193649,0.176777
1,didactiekvoorn&genn&t,0.181902,1.000000,0.166924,0.336336,0.235294,0.294118,0.177123,0.190261,0.117647,...,0.171499,0.130682,0.162698,0.235294,0.155126,0.187867,0.211702,0.171499,0.313112,0.285831
2,seriousgaming-optie2*,0.229416,0.166924,1.000000,0.190885,0.111283,0.278207,0.167542,0.134976,0.222566,...,0.270369,0.206021,0.051299,0.445132,0.146735,0.355409,0.250313,0.108148,0.236940,0.270369
3,advancedcomputervision,0.416025,0.336336,0.190885,1.000000,0.201802,0.269069,0.202548,0.217571,0.134535,...,0.196116,0.199254,0.248069,0.269069,0.118262,0.358057,0.242091,0.196116,0.358057,0.326860
4,"kind,kraam,jeugd",0.181902,0.235294,0.111283,0.201802,1.000000,0.117647,0.177123,0.190261,0.176471,...,0.228665,0.087121,0.325396,0.058824,0.206835,0.125245,0.264628,0.285831,0.125245,0.228665


In [141]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# --- CSV preprocessing (as before) ---
column_names = ['name', 'shortdescription', 'description', 'content', 'learningoutcomes', 'studycredit', 'location', 'level', 'intrest_match_score', 'popularity_score','estimated_difficulty', 'available_spots' ]

df = pd.read_csv("Uitgebreide_VKM_dataset_cleaned2.csv", sep=';', encoding='latin-1', header=None, names=column_names, engine='python', quotechar='"', on_bad_lines='skip')

# Cleaning
df['name'] = df['name'].str.lower().str.strip()
df['description'] = df['description'].str.lower().str.replace(' ', '')
df['learningoutcomes'] = df['learningoutcomes'].str.lower().str.strip()

# Combine columns
df['data'] = df[['name', 'description', 'learningoutcomes']].apply(lambda x: ' '.join(x.dropna()), axis=1)

# Vectorization
vectorizer = CountVectorizer()
vectorized = vectorizer.fit_transform(df['data'])

# Input book
# input_book = 'far beyond the stars (star trek deep space nine)'
input_processed = userInput.lower().strip()

# Vectorize the input
input_vector = vectorizer.transform([input_processed])

# Compute cosine similarity
similarities = cosine_similarity(input_vector, vectorized).flatten()

# Top 10 books with similarity score
top_indices = similarities.argsort()[::-1][0:5]
top_books = df.iloc[top_indices][['name']].copy()
top_books['similarity_score'] = similarities[top_indices]

print(top_books)
# show full name of number 1
print("Most similar book:", top_books.iloc[0]['name'])


                                                  name  similarity_score
255  doe dan mee aan een onderzoeksproject naar keu...          0.397953
254  doe dan mee aan een onderzoeksproject naar keu...          0.397953
253  doe dan mee aan een onderzoeksproject naar keu...          0.397953
358  condities creã«ren om te interveniã«ren: hoe k...          0.387147
408  384,development for storytellingâ ,"stopmotion...          0.381882
Most similar book: doe dan mee aan een onderzoeksproject naar keuze in een lectoraat in het avans zwaartepunt transities in energie en materiaal! ","vind jij het ook belangrijk om positief bij te dragen aan de maatschappij voor jezelf en toekomstige generaties? en wil je ook nog jouw onderzoeksskills verbeteren? heb je nog een minor te kiezen of 15 studiepunten openstaan van je stage of een eps-traject?


In [142]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Load the Iris dataset
# data = load_iris()
# remove non-numeric columns for simplicity

X = df_test.select_dtypes(include=['int64', 'float64']) # Selecting only numeric columns
y = df_test['name']  # Target: the module/course


# display(X.head())
# display(y.head())
# Preprocess the data (scale the features)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Set the learning rate, max_iter, and hidden layer sizes as specified
learning_rate = 0.01
max_iter = 100 # from 5 to 40
hidden_layer_sizes = 3

# Initialize the MLPClassifier with the given hyperparameters
model = MLPClassifier(
    hidden_layer_sizes=hidden_layer_sizes,
    learning_rate_init=learning_rate,
    max_iter=max_iter,
    random_state=42 
)

# Train the model
model.fit(X_train, y_train)

# Predict on the training set
y_train_pred = model.predict(X_train)

# Predict on the test set
y_test_pred = model.predict(X_test)

# Calculate accuracy for both training and test sets
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Print the results
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


c:\Users\Dhr. Kootwijk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:302: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  self.y_type_ = type_of_target(y, input_name="y")
c:\Users\Dhr. Kootwijk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  ys_types = set(type_of_target(x) for x in ys)
c:\Users\Dhr. Kootwijk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:359: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  y_is_multilabel = type_of_target(y).startswith("multilabel")
c:\Users\

Training Accuracy: 0.1636
Test Accuracy: 0.0000


In [ ]:
# Example user input
user_input = [[5, 3]]  # Must be a 2D array

# Scale the input using the same scaler
user_input_scaled = scaler.transform(user_input)

# Predict the module
predicted_module = model.predict(user_input_scaled)

print(f"The best module for you is: {predicted_module[0]}")


c:\Users\Dhr. Kootwijk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: X has 2 features, but StandardScaler is expecting 7 features as input.

: 

In [ ]:
# predict using user input
